In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [204]:
spark=SparkSession.builder.appName('Sptfy').getOrCreate()

In [205]:
df = spark.read.option("header", True) .csv("spotify.csv")

In [206]:
df.show()

+--------------------+--------------------+--------------------+-----------+------------+----+------------+------------+-------+----------------+--------+--------+-----------+-------+-------+----+---+----------+--------+
|                  id|                name|             artists|duration_ms|release_date|year|acousticness|danceability| energy|instrumentalness|liveness|loudness|speechiness|  tempo|valence|mode|key|popularity|explicit|
+--------------------+--------------------+--------------------+-----------+------------+----+------------+------------+-------+----------------+--------+--------+-----------+-------+-------+----+---+----------+--------+
|6KbQ3uYMLKb5jDxLF...|Singende Bataillo...| ['Carl Woitschach']|     158648|        1928|1928|       0.995|       0.708|  0.195|           0.563|   0.151| -12.428|     0.0506|118.469|  0.779|   1| 10|         0|       0|
|6KuQTIu1KoTTkLXKr...|Fantasiestücke, O...|['Robert Schumann...|     282133|        1928|1928|       0.994|       0.

In [ ]:
df=df.withColumn('duration_min',f.round(df.duration_ms/60000,2))\
    .withColumn("artists", f.regexp_replace("artists", r"\[|\]|'", ""))\
    .withColumn('release_date',f.when(f.length(df.release_date)=='4',f.concat(f.lit('1/1/'),f.right(df.release_date,f.lit(2)))).otherwise(df.release_date))
   

df.show()

+--------------------+--------------------+--------------------+-----------+------------+----+------------+------------+-------+----------------+--------+--------+-----------+-------+-------+----+---+----------+--------+------------+
|                  id|                name|             artists|duration_ms|release_date|year|acousticness|danceability| energy|instrumentalness|liveness|loudness|speechiness|  tempo|valence|mode|key|popularity|explicit|duration_min|
+--------------------+--------------------+--------------------+-----------+------------+----+------------+------------+-------+----------------+--------+--------+-----------+-------+-------+----+---+----------+--------+------------+
|6KbQ3uYMLKb5jDxLF...|Singende Bataillo...|     Carl Woitschach|     158648|      1/1/28|1928|       0.995|       0.708|  0.195|           0.563|   0.151| -12.428|     0.0506|118.469|  0.779|   1| 10|         0|       0|        2.64|
|6KuQTIu1KoTTkLXKr...|Fantasiestücke, O...|Robert Schumann, ...|

In [215]:
df1=df.groupBy("artists","year")\
    .agg(f.count('name').alias('Count'))\
    .sort(f.count('name'),ascending=False)
df1.show()

+--------------------+----+-----+
|             artists|year|Count|
+--------------------+----+-----+
|   Эрих Мария Ремарк|1945|  536|
|    Эрнест Хемингуэй|1940|  391|
|    Эрнест Хемингуэй|1935|  277|
|   Эрих Мария Ремарк|1929|  245|
|  Orchestra Studio 7|1942|  238|
|    Francisco Canaro|1927|  227|
|    Эрнест Хемингуэй|1926|  219|
|Sinclair Lewis, F...|1935|  212|
|    Эрнест Хемингуэй|1929|  180|
|Frédéric Chopin, ...|1946|  151|
|Frédéric Chopin, ...|1928|  149|
|    Francisco Canaro|1929|  134|
|Francisco Canaro,...|1928|  126|
|         The Beatles|1967|  120|
|Franz Joseph Hayd...|1935|  116|
|Francisco Canaro,...|1930|  115|
|Francisco Canaro,...|1929|  114|
|Igor Stravinsky, ...|1948|  112|
|Billie Holiday, T...|1933|  110|
|      Billie Holiday|1933|  110|
+--------------------+----+-----+
only showing top 20 rows



In [208]:
df.write.format("parquet").mode("overwrite").save("spty")

#df2.write.format("csv").mode('overwrite').save("/tmp/spark_output/zipcodes")